## Importing necessary packages and data

In [1]:
import numpy as np
import pandas as pd
from pandas import IndexSlice
from scipy.optimize import newton

In [2]:
# Market data as of Dec 29, 2023
summary = pd.DataFrame(index=[],columns = [207391,204095],dtype=float)
summary.loc['issue date'] = ['2019-08-15','1999-08-15']
summary.loc['maturity date'] = ['2029-08-15','2029-08-15']
summary.loc['coupon rate'] = [.01625, .06125]
summary.loc['clean price'] = [89.03125,111.0391]
summary.loc['accrued interest'] = [.6005, 2.2636]
summary.loc['ytm'] = [.037677, .038784]

summary.style.format("{:.2%}", subset=IndexSlice[["ytm","coupon rate"], :]).format("{:.2f}", subset=IndexSlice[["accrued interest","clean price"], :])

,207391,204095
issue date,2019-08-15,1999-08-15
maturity date,2029-08-15,2029-08-15
coupon rate,1.62%,6.12%
clean price,89.03,111.04
accrued interest,0.60,2.26
ytm,3.77%,3.88%


<br><br>
 _____

## 1.1 Trade Identification

- **Long:** T-bond (ID: 204095)  
- **Short:** T-note (ID: 207391)

This trade is based on a yield spread between two U.S. Treasury securities that share the same maturity date (**2029-08-15**) and identical credit quality, but have different coupons.

**Yields:**
- **T-note YTM:** 3.77%
- **T-bond YTM:** 3.88%

In an efficient market, securities with the same maturities and risk profiles should trade at similar yields. In this case, the T-bond trades at a higher yield (3.88%) than the T-note (3.77%), which means the T-bond is considered "cheap" (undervalued) relative to the T-note, which is "rich" (overvalued).

To exploit this arbitrage opportunity:
- **Buy** the security with the higher yield (T-bond)
- **Sell** the security with the lower yield (T-note)

This position profits from an expected convergence of yields (and prices) over time. <br><br>
 _____

## 1.2 Financing the Trade

To execute this trade with minimal upfront capital, you would use the **repo market** (Repurchase Agreement market). The trade involves two distinct financing legs:

### 1. Financing the Long Position (T-bond)

- **Mechanism:** Enter into a standard **repo** agreement.
- **Process:** 
    - Buy the T-bond in the cash market.
    - Simultaneously, "sell" the bond to a dealer (repo counterparty) with an agreement to repurchase it at a later date for a slightly higher price.
- **Result:** 
    - The cash received from the dealer funds your purchase of the bond.
    - You are effectively borrowing money using the T-bond as collateral, and pay the **repo rate** (the financing cost) on the cash borrowed.

### 2. Sourcing the Short Position (T-note)

- **Mechanism:** Enter into a **reverse repo** (or securities lending agreement).
- **Process:** 
    - To short the T-note, first borrow the security.
    - Lend cash to a dealer (reverse repo) and receive the T-note as collateral.
    - Sell the borrowed T-note in the open market to establish your short position.
- **Result:** 
    - The cash proceeds from the short sale are typically used to fund the cash lending in the reverse repo.
    - You earn interest (**rebate rate**) on the cash lent to the dealer.

<br>

**Net Mechanics:**  
In a frictionless world, the proceeds from the short sale fund the long purchase. In reality, you must deal with "haircuts" (margin), so a small percentage of equity capital is required. You **pay the repo rate** on the long leg and **earn the reverse repo rate** on the short leg. <br><br>
 _____

## 1.3 Risks and Arbitrage Horizon

#### Is this an Arbitrage?

- **Short-term:**  
  **No.** In the short term, this is a risky relative value trade—not a risk-free arbitrage. Market noise, liquidity constraints, or supply/demand imbalances can cause the spread (3.88% vs 3.77%) to widen rather than narrow, leading to mark-to-market losses. Additionally, the short position (T-note) risks going **"special"** in the repo market (where borrowing costs spike), which would erode profits.

- **Long-term:**  
  **Yes (approximately).** This is a convergence trade. At maturity (**Aug 15, 2029**), both bonds will redeem at Par ($100). The price difference must disappear, and the yields must converge. Therefore, if you can hold the position to maturity (and manage the financing/carry costs), the prices will align.

<br>

#### Key Risks

- **Convergence Risk:**  
  The spread may widen before it narrows.

- **Financing Risk (Repo Risk):**  
  The cost to borrow the T-note (short) could skyrocket if the note becomes "special" (hard to locate).

- **Carry Risk:**  
  The difference in coupons (6.12% vs 1.62%) creates a "negative carry" on the price principal if financing rates are high, though the current yield spread suggests positive carry.

- **Basis Risk:**  
  While maturity is the same, small differences in liquidity or deliverability (into futures) can sustain price gaps. <br><br>
 _____

## 1.4 Calculate New YTM

We solve for the Yield to Maturity ($y$) using the standard bond pricing formula.
- **Settlement Date:** 2024-02-15  
- **Maturity Date:** 2029-08-15  
- **Time to Maturity:** 5.5 Years ($N = 11$ periods, semiannual)
- **Prices:**  
   - Note = 87.00  
   - Bond = 113.00  
- **Coupons:**  
   - Note = 1.62%  
   - Bond = 6.12%

In [3]:
def calculate_ytm(price, coupon_rate, T, freq=2):
    # T is years to maturity
    # coupon_rate is annual decimal
    # price is per 100 face value
    periods = int(T * freq)
    coupon_payment = (coupon_rate * 100) / freq
    
    # Define the pricing function based on yield (y)
    def price_func(y):
        # y is the annual yield
        if y == 0:
            return (coupon_payment * periods + 100) - price
        
        # Vectorized discount factors
        t = np.arange(1, periods + 1)
        discount_factors = (1 + y/freq) ** -t
        
        val = np.sum(coupon_payment * discount_factors) + 100 * (1 + y/freq) ** -periods
        return val - price

    # Solve for y using Newton-Raphson
    # Guess 4%
    return newton(price_func, 0.04)

# Inputs
t_maturity_14 = 5.5
price_note_14 = 87.0
price_bond_14 = 113.0
coupon_note = 0.0162
coupon_bond = 0.0612

ytm_note_14 = calculate_ytm(price_note_14, coupon_note, t_maturity_14)
ytm_bond_14 = calculate_ytm(price_bond_14, coupon_bond, t_maturity_14)

# Display the results
ytm_df = pd.DataFrame({
    "Instrument": ["T-Note", "T-Bond"],
    "Yield to Maturity": [ytm_note_14, ytm_bond_14]
})
ytm_df["Yield to Maturity"] = ytm_df["Yield to Maturity"].apply(lambda x: f"{x:.5%}")
ytm_df.set_index("Instrument", inplace=True)
ytm_df

,Yield to Maturity
Instrument,
T-Note,4.29929%
T-Bond,3.50094%


## 1.5 Prices on 2024-08-15

Date: 2024-08-15  
**Time to Maturity:** 5.0 Years ($N=10$ periods)  
**YTMs:** Note = 4.65%, Bond = 4.70%  
**Condition:** Immediately after coupon payment (Clean Price = Dirty Price).

We calculate the prices using the new YTMs and compare them to the dirty prices from the original table (Dec 29, 2023).

**Original (Dec 29, 2023) Dirty Prices:**  
- T-note: $89.03$ (clean) $+~0.60$ (accrued) $=~\mathbf{89.63}$
- T-bond: $111.04$ (clean) $+~2.26$ (accrued) $=~\mathbf{113.30}$

In [4]:
def calculate_price(ytm, coupon_rate, T, freq=2):
    periods = int(T * freq)
    coupon_payment = (coupon_rate * 100) / freq
    t = np.arange(1, periods + 1)
    discount_factors = (1 + ytm/freq) ** -t
    price = np.sum(coupon_payment * discount_factors) + 100 * (1 + ytm/freq) ** -periods
    return price

# Inputs
t_maturity_15 = 5.0
ytm_note_15 = 0.0465
ytm_bond_15 = 0.0470

# Calculate New Prices
price_note_15 = calculate_price(ytm_note_15, coupon_note, t_maturity_15)
price_bond_15 = calculate_price(ytm_bond_15, coupon_bond, t_maturity_15)

# Original Dirty Prices (from table)
dirty_note_orig = 89.03 + 0.60
dirty_bond_orig = 111.04 + 2.26


# Display the results
price_comparison_df = pd.DataFrame({
    "Instrument": ["T-Note", "T-Bond"],
    "New Price (2024-08-15)": [f"{price_note_15:.4f}", f"{price_bond_15:.4f}"],
    "Original Dirty Price (2023-12-29)": [f"{dirty_note_orig:.4f}", f"{dirty_bond_orig:.4f}"]
})

price_comparison_df.set_index("Instrument", inplace=True)
price_comparison_df

,New Price (2024-08-15),Original Dirty Price (2023-12-29)
Instrument,,
T-Note,86.6199,89.6300
T-Bond,106.2624,113.3000


## 1.6 Long/Short Position Value

 We calculate the value of the portfolio: **Long 1 Unit T-Bond, Short 1 Unit T-Note**.
 
The value ($V$) of the position is:
 
$$V = \text{Price of Long Asset} - \text{Price of Short Asset}$$
 
**Value at Start (Dec 29, 2023):**
 
Using the dirty prices (actual cash exchanged):
 
$$
V_{\text{initial}} = 113.30 - 89.63 = \mathbf{23.67}
$$
 
**Value at End (Aug 15, 2024):**

Using the prices calculated in 1.5:
 
$$
V_{\text{new}} = 106.2624 - 86.6199 = \mathbf{19.64}
$$
 
**Change in Value:**
 
$$
\Delta V = 19.64 - 23.67 = \mathbf{-4.03}
$$

In [5]:
# Calculation
val_initial = dirty_bond_orig - dirty_note_orig
val_new = price_bond_15 - price_note_15
change = val_new - val_initial

# Display the results
position_value_df = pd.DataFrame({
    "Value": [val_initial, val_new, change]
}, index=["Initial Value (2023-12-29)", "New Value (2024-08-15)", "Change in Value"])

position_value_df.style.format("{:.4f}")

,Value
Initial Value (2023-12-29),23.6700
New Value (2024-08-15),19.6425
Change in Value,-4.0275
